#1.드라이브 연동 / 필요모듈설치

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install pycaret

#2.모듈 import / 데이터 경로지정

In [4]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')
PATH = '/content/drive/MyDrive/LG AI대회/'

#3.학습(train_X, test_X) 데이터 전처리 함수 만들기

In [5]:
def make_df(train_err, quality):
  #train_err  = pd.read_csv(PATH+'train_err_data.csv')

################################################################################################################

  ###train_err데이터의 time변수를 연 월 일 변수로 나눠서 만들어주기
  train_err['year'] = train_err['time'].astype(str).apply(lambda x: int(x[:4]))
  train_err['month'] = train_err['time'].astype(str).apply(lambda x: int(x[4:6]))
  train_err['day'] = train_err['time'].astype(str).apply(lambda x: int(x[6:8]))

  train_err['hour'] = train_err['time'].astype(str).apply(lambda x: int(x[8:10]))
  train_err['min'] = train_err['time'].astype(str).apply(lambda x: int(x[10:12]))
  train_err['sec'] = train_err['time'].astype(str).apply(lambda x: int(x[12:14]))
  train_err['date'] = train_err['time'].astype(str).apply(lambda x: int(x[4:8]))

################################################################################################################

  ###err_type 
  pivot = pd.pivot_table(train_err, index = ['user_id'],columns = 'errtype', values = ['time'],aggfunc='count',fill_value=0)
  df_errtype = pd.DataFrame(pivot.to_records())

  df_errtype = df_errtype.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","errtype")) 

################################################################################################################

  ###err_day 
  pivot2 = pd.pivot_table(train_err, index = ['user_id'],columns = 'date', values = ['time'],aggfunc='count',fill_value=0)
  df_errday = pd.DataFrame(pivot2.to_records())

  df_errday = df_errday.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","day")) 

  #누적하는 코드
  for i in list(range(1,33)) :
          df_errday.iloc[:,i+1] = df_errday.iloc[:,i+1] + df_errday.iloc[:,i]

################################################################################################################

  ###quality
  #quality = pd.read_csv(PATH+'train_quality_data.csv')
  quality.fillna(0) #quality_5변수에 NA값이 있음

  #quality의 문자열변수,정수형변수 -> 실수형으로 바꿔주기
  cols = ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
          'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']
  quality[cols] = quality[cols].replace({',':''}, regex= True) 
  quality.iloc[:,3:16] = quality.iloc[:,3:16].astype(float) #quality_n 변수들 실수형으로 바꿔주기

  #quality5만들기(quality의 일부 + -1의 개수 뽑아냄)

  #minusQ 리스트는 각 user_id별 -1의 빈도수가 얼마나 나오는지 알기위해서 만드는 리스트임
  #현재는 모든 quality 로그가 기록될때 -1인 퀄리티가 발생하면 minuQ리스트에 -1을 아니라면 0을 할당하는 식으로 새로운 열을 만듦
  minusQ = []
  for i in range(0,len(quality)): #데이터를 보면 quality 0,1,2,5,6,11 번째에서 동시에 -1이 발발하는것을 알 수 있었음.
    if (quality.iloc[i,3] == -1 or quality.iloc[i,4] == -1 or quality.iloc[i,5] == -1 or quality.iloc[i,8] == -1 or quality.iloc[i,9] == -1 or quality.iloc[i,14] == -1) :
    #if (quality.iloc[i,3] == -1 and quality.iloc[i,4] == -1 and quality.iloc[i,5] == -1 and quality.iloc[i,8] == -1 and quality.iloc[i,9] == -1 and quality.iloc[i,14] == -1) :
      minusQ.append(-1)
    else:
      minusQ.append(0)

  quality['minusQ'] = minusQ #minusQ 열 추가해주기

  #규칙을 찾아보면 quality는 무언가 누적합의 경우가 많음 그래서 가장 큰 숫자(max)를 대표값으로 지정하도록 코딩하였음
  #minusQ의 경우는 -1의 발생 빈도를 볼 것이기 때문에, 예외적으로 count를 볼 것이기 떄문에 합(sum)으로 코딩하였음

  def func(x):
      d = {}
      d['quality_0'] = x['quality_0'].max()
      d['quality_1'] = x['quality_1'].max()
      d['quality_2'] = x['quality_2'].max()
      d['quality_3'] = x['quality_3'].max()
      d['quality_4'] = x['quality_4'].max()
      d['quality_5'] = x['quality_5'].max()
      d['quality_6'] = x['quality_6'].max()
      d['quality_7'] = x['quality_7'].max()
      d['quality_8'] = x['quality_8'].max()
      d['quality_9'] = x['quality_9'].max()
      d['quality_10'] = x['quality_10'].max()
      d['quality_11'] = x['quality_11'].max()
      d['quality_12'] = x['quality_12'].max()
      d['minusQ'] = x['minusQ'].sum()
      return pd.Series(d, index= ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
                                  'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12', 'minusQ'])

  quality5 = quality.groupby(['user_id','time']).apply(func)

  quality5 = quality5.groupby(['user_id']).sum()
  quality5 = quality5.drop(columns = ['quality_3', 'quality_4']) 

################################################################################################################

  ###errcode
  pivot3 = pd.pivot_table(train_err, index = ['user_id'],columns = 'errcode', values = ['time'],aggfunc='count',fill_value=0)
  df_errcode = pd.DataFrame(pivot3.to_records())

  df_errcode = df_errcode.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","errcode_"))

################################################################################################################

  #위에서 만든 데이터들 하나로 합병(join&merge)
  df_result = pd.merge(df_errtype, df_errday, on='user_id', how='inner')
  df_result = pd.merge(df_result, quality5, left_on='user_id', right_on='user_id', how='left')
  df_result.fillna(0, inplace = True)  
  df_result = pd.merge(df_result, df_errcode, left_on='user_id', right_on='user_id', how='left')
  df_result.fillna(0, inplace = True)

  return df_result

In [6]:
'''
함수를 한번에 안하고 변수별로 나누어 여러개의 함수로 만들었음
################################################################################################################

def make_errtype(train_err):
  #train_err  = pd.read_csv(PATH+'train_err_data.csv')

  ###err_type 
  pivot = pd.pivot_table(train_err, index = ['user_id'],columns = 'errtype', values = ['time'],aggfunc='count',fill_value=0)
  df_errtype = pd.DataFrame(pivot.to_records())

  df_errtype = df_errtype.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","errtype")) 

  return df_errtype
################################################################################################################

def make_errday(train_err):
  ###train_err데이터의 time변수를 연 월 일 변수로 나눠서 만들어주기
  train_err['year'] = train_err['time'].astype(str).apply(lambda x: int(x[:4]))
  train_err['month'] = train_err['time'].astype(str).apply(lambda x: int(x[4:6]))
  train_err['day'] = train_err['time'].astype(str).apply(lambda x: int(x[6:8]))

  train_err['hour'] = train_err['time'].astype(str).apply(lambda x: int(x[8:10]))
  train_err['min'] = train_err['time'].astype(str).apply(lambda x: int(x[10:12]))
  train_err['sec'] = train_err['time'].astype(str).apply(lambda x: int(x[12:14]))
  train_err['date'] = train_err['time'].astype(str).apply(lambda x: int(x[4:8]))


  ###err_day 
  pivot2 = pd.pivot_table(train_err, index = ['user_id'],columns = 'date', values = ['time'],aggfunc='count',fill_value=0)
  df_errday = pd.DataFrame(pivot2.to_records())

  df_errday = df_errday.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","day")) 

  #누적하는 코드
  for i in list(range(1,33)) :
          df_errday.iloc[:,i+1] = df_errday.iloc[:,i+1] + df_errday.iloc[:,i]

  return df_errday
################################################################################################################


def make_quality(quality):
  ###quality
  #quality = pd.read_csv(PATH+'train_quality_data.csv')
  quality.fillna(0) #quality_5변수에 NA값이 있음

  #quality의 문자열변수,정수형변수 -> 실수형으로 바꿔주기
  cols = ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
          'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']
  quality[cols] = quality[cols].replace({',':''}, regex= True) 
  quality.iloc[:,3:16] = quality.iloc[:,3:16].astype(float) #quality_n 변수들 실수형으로 바꿔주기

  #quality5만들기(quality의 일부 + -1의 개수 뽑아냄)

  #minusQ 리스트는 각 user_id별 -1의 빈도수가 얼마나 나오는지 알기위해서 만드는 리스트임
  #현재는 모든 quality 로그가 기록될때 -1인 퀄리티가 발생하면 minuQ리스트에 -1을 아니라면 0을 할당하는 식으로 새로운 열을 만듦
  minusQ = []
  for i in range(0,len(quality)): #데이터를 보면 quality 0,1,2,5,6,11 번째에서 동시에 -1이 발발하는것을 알 수 있었음.
    if (quality.iloc[i,3] == -1 or quality.iloc[i,4] == -1 or quality.iloc[i,5] == -1 or quality.iloc[i,8] == -1 or quality.iloc[i,9] == -1 or quality.iloc[i,14] == -1) :
    #if (quality.iloc[i,3] == -1 and quality.iloc[i,4] == -1 and quality.iloc[i,5] == -1 and quality.iloc[i,8] == -1 and quality.iloc[i,9] == -1 and quality.iloc[i,14] == -1) :
      minusQ.append(-1)
    else:
      minusQ.append(0)

  quality['minusQ'] = minusQ #minusQ 열 추가해주기

  #규칙을 찾아보면 quality는 무언가 누적합의 경우가 많음 그래서 가장 큰 숫자(max)를 대표값으로 지정하도록 코딩하였음
  #minusQ의 경우는 -1의 발생 빈도를 볼 것이기 때문에, 예외적으로 count를 볼 것이기 떄문에 합(sum)으로 코딩하였음

  def func(x):
      d = {}
      d['quality_0'] = x['quality_0'].max()
      d['quality_1'] = x['quality_1'].max()
      d['quality_2'] = x['quality_2'].max()
      d['quality_3'] = x['quality_3'].max()
      d['quality_4'] = x['quality_4'].max()
      d['quality_5'] = x['quality_5'].max()
      d['quality_6'] = x['quality_6'].max()
      d['quality_7'] = x['quality_7'].max()
      d['quality_8'] = x['quality_8'].max()
      d['quality_9'] = x['quality_9'].max()
      d['quality_10'] = x['quality_10'].max()
      d['quality_11'] = x['quality_11'].max()
      d['quality_12'] = x['quality_12'].max()
      d['minusQ'] = x['minusQ'].sum()
      return pd.Series(d, index= ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
                                  'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12', 'minusQ'])

  quality5 = quality.groupby(['user_id','time']).apply(func)

  quality5 = quality5.groupby(['user_id']).sum()
  quality5 = quality5.drop(columns = ['quality_3', 'quality_4']) 

  return quality5
################################################################################################################

def make_errcode(train_err)
  ###errcode
  pivot3 = pd.pivot_table(train_err, index = ['user_id'],columns = 'errcode', values = ['time'],aggfunc='count',fill_value=0)
  df_errcode = pd.DataFrame(pivot3.to_records())

  df_errcode = df_errcode.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","errcode_"))

  return df_errcode
################################################################################################################
'''

'\n함수를 한번에 안하고 변수별로 나누어 여러개의 함수로 만들었음\n################################################################################################################\n\ndef make_errtype(train_err):\n  #train_err  = pd.read_csv(PATH+\'train_err_data.csv\')\n\n  ###err_type \n  pivot = pd.pivot_table(train_err, index = [\'user_id\'],columns = \'errtype\', values = [\'time\'],aggfunc=\'count\',fill_value=0)\n  df_errtype = pd.DataFrame(pivot.to_records())\n\n  df_errtype = df_errtype.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("\'","").replace(" ","").replace("time","errtype")) \n\n  return df_errtype\n################################################################################################################\n\ndef make_errday(train_err):\n  ###train_err데이터의 time변수를 연 월 일 변수로 나눠서 만들어주기\n  train_err[\'year\'] = train_err[\'time\'].astype(str).apply(lambda x: int(x[:4]))\n  train_err[\'month\'] = train_err[\'time\'].astype(str).apply(lambda x: int(x[4:6]))\n  t

##3_1.Train data 불러오기/함수적용

In [7]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_quality = pd.read_csv(PATH+'train_quality_data.csv')
X_train = make_df(train_err, train_quality)

KeyboardInterrupt: ignored

##3_2.Test data 불러오기/함수적용

In [ ]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
test_quality = pd.read_csv(PATH+'test_quality_data.csv')
X_test = make_df(test_err, test_quality)

#4.(추가전처리) 

##Train_X, Test_X 특징

##4_1. err_type29에 해당하는 사람없음

##4_2. 10월 12월은 발생한 에러가 0이 대부분이므로 제외하겠음

##4_3. Test_X의 43262유저는 에러기록이 한번도 없으므로 전부 0으로 추가하도록 하겠음
(quality에서는 43262유저의 기록이 나타나서 qulity관련변수까지0으로 취급이 되어버리는 문제가 발생함)

##4_4. X_train X_test 공통변수 추출

##4_5. 중간 전처리완료파일 저장

##4_2. Train_X, Test_X 에 10월 12월 없애주기

In [ ]:
X_train.drop(columns = ['day1031','day1201','day1202'], inplace = True)
X_test.drop(columns = ['day1031','day1201','day1202','day1203','day1204','day1205','day1209','day1211','day1214'], inplace = True)

##4_3. Test_X에 43262 유저 추가해주기


In [ ]:
X_test.loc[14998] = 0
X_test.loc[14998,'user_id'] = 43262
X_test.set_index(['user_id'], inplace=True)
X_test.sort_index(inplace=True)

##4_4.X_train과 X_test의 공통 변수만을 뽑아내 재정의해주기

In [ ]:
inter_cols = X_train.columns & X_test.columns
X_train = X_train[inter_cols]
X_test = X_test[inter_cols]

In [ ]:
X_train

In [ ]:
X_test

##4_5.중간에 Train_X Test_X 파일로 저장

In [8]:
#X_train.to_csv("X_train.csv", index = False)
#X_test.to_csv("X_test.csv", index = False)

#불러오기
#X_train = pd.read_csv('/content/X_train.csv')
#X_test = pd.read_csv('/content/X_test.csv')

#5.Train_Y + Train_X 만들기 


In [9]:
#Train_Y 만들기
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
problem = np.zeros(15000)

problem[train_prob.user_id.unique()-10000] = 1 

Y_train = pd.DataFrame(problem)
Y_train

,0
0,0.0
1,1.0
2,0.0
3,0.0
4,1.0
...,...
14995,0.0
14996,0.0
14997,1.0
14998,1.0


In [10]:
master_data = X_train
master_data['problem'] = problem

#AUTOML돌리기 위해서 데이터타입을 실수형으로 변환
cols = master_data.columns
master_data = master_data[cols].astype('float')

#6.AUTO ML 돌리기

In [11]:
clf = setup(data = master_data, target = 'problem')

,Description,Value
0,session_id,4747
1,Target,problem
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 1491)"
5,Missing Values,False
6,Numeric Features,397
7,Categorical Features,1093
8,Ordinal Features,False
9,High Cardinality Features,False


In [12]:
best_n = compare_models(sort = 'Accuracy', n_select = 7)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.7887,0.8207,0.5097,0.7812,0.6164,0.4795,0.5006,49.372
lightgbm,Light Gradient Boosting Machine,0.7867,0.8130,0.5249,0.7617,0.6211,0.4798,0.4961,1.064
gbc,Gradient Boosting Classifier,0.7841,0.8105,0.4786,0.7917,0.5961,0.4609,0.4883,14.479
et,Extra Trees Classifier,0.7826,0.8040,0.4715,0.7932,0.5907,0.4558,0.4846,8.686
xgboost,Extreme Gradient Boosting,0.7770,0.7997,0.5423,0.7202,0.6185,0.4654,0.4749,43.445
rf,Random Forest Classifier,0.7766,0.8038,0.4638,0.7766,0.5802,0.4412,0.4686,4.500
ada,Ada Boost Classifier,0.7762,0.7956,0.4846,0.7566,0.5904,0.4466,0.4678,3.712
ridge,Ridge Classifier,0.7441,0.0000,0.3467,0.7510,0.4733,0.3342,0.3779,1.842
lda,Linear Discriminant Analysis,0.7423,0.7268,0.3584,0.7309,0.4799,0.3351,0.3728,6.184
dt,Decision Tree Classifier,0.6882,0.6538,0.5506,0.5320,0.5408,0.3050,0.3052,0.735


In [13]:
blended = blend_models(estimator_list = best_n, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7757,0.8051,0.4736,0.7650,0.5850,0.4428,0.4667
1,0.7976,0.8320,0.5435,0.7840,0.6420,0.5073,0.5238
2,0.7824,0.8104,0.4843,0.7793,0.5974,0.4592,0.4836
3,0.7919,0.8206,0.5000,0.8009,0.6157,0.4832,0.5085
4,0.7918,0.8245,0.5000,0.8009,0.6157,0.4832,0.5084
Mean,0.7879,0.8185,0.5003,0.7860,0.6111,0.4751,0.4982
SD,0.0078,0.0097,0.0238,0.0137,0.0193,0.0222,0.0204


In [14]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7894,0.8219,0.496,0.7938,0.6105,0.4765,0.5013


In [15]:
final_model = finalize_model(blended)

#7.예측(Prediction)

In [16]:
predictions = predict_model(final_model, data = X_test)

In [17]:
predictions

,errtype1,errtype2,errtype3,errtype4,errtype5,errtype6,errtype7,errtype8,errtype9,errtype10,errtype11,errtype12,errtype13,errtype14,errtype15,errtype16,errtype17,errtype18,errtype19,errtype20,errtype21,errtype22,errtype23,errtype24,errtype25,errtype26,errtype27,errtype28,errtype30,errtype31,errtype32,errtype33,errtype34,errtype35,errtype36,errtype37,errtype38,errtype39,errtype40,errtype41,...,errcode_P-44010,errcode_P-44010.1,errcode_PHONE_ERR,errcode_PUBLIC_ERR,errcode_P_41001,errcode_Q-64001,errcode_Q-64002,errcode_Q-73004,errcode_Q-73006,errcode_S-61001,errcode_S-64000,errcode_S-64001,errcode_S-64002,errcode_S-65,errcode_S-65002,errcode_U-81000,errcode_U-81009,errcode_U-81014,errcode_U-82004,errcode_U-82020,errcode_U-82023,errcode_U-82026,errcode_UNKNOWN,errcode_V-21002,errcode_V-21003,errcode_V-21004,errcode_V-21005,errcode_V-21007,errcode_V-21008,errcode_Y-00008,errcode_active,errcode_connectionfailforLMPresponsetimout,errcode_connectionfailtoestablish,errcode_connectionerrcode_out,errcode_connectionterminatedbylocalhost,errcode_scanningerrcode_out,errcode_standby,errcode_terminatebypeeruser,Label,Score
0,0,0,0,0,62,1,1,0,0,0,16,16,0,3,67,37,9,3,0,1,0,983,1308,5,2,32,0,0,0,83,2,16,0,1,3,3,0,2,92,0,...,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,141,0,55,1107,0,0,4,0,1.0,0.8556
1,0,0,3,98,3,4,4,0,0,0,12,16,4,6,53,55,0,0,0,0,0,0,0,0,0,24,1,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.5435
2,0,0,0,0,79,2,1,0,0,0,16,16,0,1,173,133,0,0,0,0,0,60,40,2,0,40,0,0,0,230,0,16,0,0,1,1,0,14,113,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,7,0,0,26,0,0,6,1,1.0,0.6814
3,0,0,0,0,107,2,2,0,0,0,4,5,1,7,8,12,0,1,0,1,0,3,2,0,0,5,2,2,0,134,0,4,16,3,1,1,0,0,40,8,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1.0,0.8152
4,0,0,0,2,6,1,1,0,0,0,10,12,1,18,80,67,6,0,0,0,0,10,8,0,0,14,1,1,0,332,152,11,6,1,1,1,1,0,48,90,...,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,3,0,1.0,0.7791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,0,0,0,2,12,5,5,0,0,0,16,19,3,9,123,93,12,0,0,0,0,197,204,0,0,31,0,0,0,179,2,16,0,0,0,0,0,0,100,87,...,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,197,0,0,0,0,0.0,0.5292
14995,0,0,0,0,0,2,2,0,0,0,14,15,1,4,149,98,0,0,0,0,0,0,0,0,0,1,0,0,0,183,0,14,1,0,1,1,0,0,12,17,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.5913
14996,0,0,0,0,0,1,1,0,0,0,12,13,1,3,129,90,2,0,0,0,0,138,165,0,0,48,0,0,0,163,0,12,1118,0,1,1,1,0,329,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,104,0,4,44,0,0,11,2,1.0,0.8020
14997,0,0,0,1,4,2,2,0,0,0,15,16,0,1,73,44,0,1,0,1,0,11127,13161,0,3,44,0,0,0,78,0,14,0,0,1,1,3,0,30,49,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,25,0,0,0,0,0,0,0,1655,0,1010,10449,1,3,20,1,1.0,0.8094


#8.제출파일(Y_test) 생성

In [20]:
#pycaret에서는 score이 label을 맞출 확률이기때문에 output을 제출 양식에 맞게 바꿔줍니다
x = []
for i in range(0,14999):
  if predictions['Label'][i] =='1.0':
    x.append(predictions['Score'][i])
  else:
    x.append(1-predictions['Score'][i])

In [21]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')
sample_submssion['problem'] = x
sample_submssion.to_csv("AutoML.csv", index = False)